In [1]:
%pip install mosestokenizer

  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Created wheel for mosestokenizer: filename=mosestokenizer-1.2.1-py3-none-any.whl size=49170 sha256=34ba537c1953fb88ec8fcc84a2574542ff32bbbe5dec1d8d2d8e74408f1c683e
  Stored in directory: /root/.cache/pip/wheels/80/d8/15/4c5ebbe883513f003cb055a0369c77c9df857023a706f39e70
  Created wheel for toolwrapper: filename=toolwrapper-2.1.0-py3-none-any.whl size=3338 sha256=afd656b72fec397f39ee1e781e79e025f62bb055c4bfff7166b9126782edc121
  Stored in directory: /root/.cache/pip/wheels/e1/af/b1/99b57a06dda78fdcee86d2e22c64743f3b8df8f31cfc04baf7
  Created wheel for uctools: filename=uctools-1.3.0-py3-none-any.whl size=6147 sha256=19d9c78beff3f93c8a265242ae6b486e03b8afe5a090aae8d0522667d28fd1b5
  Stored in directory: /root/.cache/pip/wheels/05/ee/10/33257b0801ac6a912c841939032c16da1eb3db377afe1443e5
Successfully built mosestokenizer toolwrapper uctools
Note: you may 

In [2]:
!pwd

/kaggle/working


In [3]:
HTML = "https://www.gutenberg.org/cache/epub/2814/pg2814-images.html"

In [4]:
import requests
from bs4 import BeautifulSoup

In [5]:
dubliners = requests.get(HTML)

In [6]:
assert dubliners.status_code == 200

In [7]:
soup = BeautifulSoup(dubliners.text, 'html.parser')

In [8]:
body = soup.find("body")

In [9]:
text = []
for chapter in body.findAll("div", {"class": "chapter"}):
    for element in chapter.findChildren():
        if element.name == "section" and element.get("id") in ["pg-header", "pg-footer"]:
            continue
        stripped = element.text.strip()
        if stripped != "":
            text.append(stripped)

In [10]:
from mosestokenizer import MosesSentenceSplitter

In [11]:
sents = []
try:
    with MosesSentenceSplitter('en') as splitsents:
        for para in text:
          if para == "":
              continue
          sents += splitsents([para.replace("\r\n", " ")])
except Exception as ex:
    print(ex, para)

In [12]:
import re
def cleaner(text):
    text = text.replace("“", "")
    text = text.replace("”", "")
    text = text.replace("’ ", " ")
    text = text.replace("’", "'")
    text = text.replace("‘", " ")
    text = text.replace("\t", " ")
    text = text.replace("!...", " ")
    text = text.replace("....", " ")
    text = text.replace("...", " ")
    text = text.replace(":", " ")
    text = text.replace(";", " ")
    text = text.replace("!", " ")
    text = text.replace(",", " ")
    text = text.replace("?", " ")
    text = text.replace("(", " ")
    text = text.replace(")", " ")
    text = text.replace("—", " ")
    text = text.replace("\r\n", " ")
    text = text.replace("\n", " ")
    text = text.replace("\xa0", " ")
    text = text.replace(".", " ")
    text = text.replace("&", " and ")
    text = text.replace(" 57E ", " fifty seven e ")
    text = text.replace(" 1st ", " first ")
    text = text.replace(" 6th ", " sixth ")
    text = text.replace(" 1895 ", " eighteen ninety five ")
    text = text.replace(" 1891", " eighteen ninety one")
    text = text.replace("1891", " eighteen ninety one")
    text = text.replace("65 ", " sixty five ")
    text = text.replace("Rev. James", "Reverend James")
    text = text.replace("S. Catherine's", "Saint Catherine's")
    if text[-1] == ".":
        text = text[:-1]
    text = re.sub("  +", " ", text)
    return text.lower().strip()

In [13]:
clean = [cleaner(x) for x in sents]

In [14]:
with open("dubliners-clean.txt", "w") as outf:
    for line in clean:
        outf.write(line + "\n")

In [15]:
%capture
!apt install -y build-essential libboost-all-dev cmake zlib1g-dev libbz2-dev liblzma-dev

UsageError: Line magic function `%capture` not found (But cell magic `%%capture` exists, did you mean that instead?).


In [ ]:
%cd /tmp

In [ ]:
!git clone https://github.com/kpu/kenlm

In [ ]:
%cd kenlm

In [ ]:
%capture
!mkdir build
%cd build
!cmake ..
!make -j 4

In [ ]:
%cd /kaggle/working

In [ ]:
!/tmp/kenlm/build/bin/lmplz -o 5 --text dubliners-clean.txt --arpa dubliners.arpa --discount_fallback

In [ ]:
!cat dubliners-clean.txt | tr ' ' '\n'|sort|uniq > unigrams.txt

In [ ]:
!/tmp/kenlm/build/bin/build_binary -a 255 -q 8 dubliners.arpa dubliners.klm